In [1]:
import os

In [10]:
last_save_id = max([int(file_name.split('_')[1]) for file_name in os.listdir('params')])
new_save_id = last_save_id + 1

In [16]:
config = {
    'iterations':50,
    
    'simclr_iterations':50,
    'simclr_bs':256,
    
    'ppo_iterations':100,
    'ppo_len_trajectory':512*4,
    'ppo_collection_bs':512*2,
    'ppo_update_epochs':4,
    'ppo_update_bs':256,
    
    'logs':True
}

[f'{k}={v}' for (k, v) in config.items()]

['iterations=50',
 'simclr_iterations=50',
 'simclr_bs=256',
 'ppo_iterations=100',
 'ppo_len_trajectory=2048',
 'ppo_collection_bs=1024',
 'ppo_update_epochs=4',
 'ppo_update_bs=256',
 'logs=True']